In [11]:
import rasterio


# <<< FERTILISER >>>
# The dataset consists of maps of the predicted average annual application rates (2010-2015) of three different inorganic 
# chemical fertilisers – nitrogen (N), phosphorus (P) and potassium (K) – in England across a six-year period, along with 
# their respective estimates of uncertainty, at a 1 km x 1 km resolution. 
FERTILISER_K_PATH = 'Datasets/Digimap/Land-Cover-plus-Fertilisers_4731497/fertiliser_k_prediction_uncertainty.tif'
FERTILISER_N_PATH = 'Datasets/Digimap/Land-Cover-plus-Fertilisers_4731497/fertiliser_n_prediction_uncertainty.tif'
FERTILISER_P_PATH = 'Datasets/Digimap/Land-Cover-plus-Fertilisers_4731497/fertiliser_p_prediction_uncertainty.tif'

dataset = rasterio.open(FERTILISER_P_PATH)
print(dataset.block_shapes)





[(128, 651), (128, 651)]
